In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [3]:
spark.version

'3.5.5'

question 1: 3.5.5

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet

--2025-03-16 20:36:11--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-10.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.5, 18.160.201.50, 18.160.201.126, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64346071 (61M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2024-10.parquet’

yellow_tripdata_202 100%[===================>]  61.36M  88.8MB/s    in 0.7s    

2025-03-16 20:36:12 (88.8 MB/s) - ‘yellow_tripdata_2024-10.parquet’ saved [64346071/64346071]



In [65]:
df_yellow = spark.read.parquet('yellow_tripdata_2024-10.parquet')

In [16]:
df_yellow \
  .repartition(4) \
  .write.parquet('drive/MyDrive/hw5', mode='overwrite')

question 2: 23mb

In [23]:
df_yellow.registerTempTable('yellow')

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [24]:
df_yellow_count = spark.sql("""
SELECT
    count(1)
FROM
    yellow
WHERE
    DATE(tpep_pickup_datetime) = '2024-10-15'
""")

In [39]:
df_yellow_count.show()

+--------+
|count(1)|
+--------+
|  128893|
+--------+



Question 3: 128893

In [47]:
from pyspark.sql import functions as F


In [48]:
df_yellow_with_duration = df_yellow.withColumn(
    "trip_duration_hours",
    (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 3600

)

In [50]:
longest_trip = df_yellow_with_duration.orderBy(F.desc("trip_duration_hours")).limit(1)
longest_trip.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|trip_duration_hours|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+-------------------+
|       2| 2024-10-16 13:03:49|  2024-10-23 07:40:53|              1|        32.37|         3|                 N|          48| 

Question 4: 162.617777

In [52]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv


--2025-03-16 21:18:07--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.160.201.131, 18.160.201.5, 18.160.201.126, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.160.201.131|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2025-03-16 21:18:08 (214 MB/s) - ‘taxi_zone_lookup.csv’ saved [12331/12331]



In [56]:
df_zones = spark.read.parquet('drive/MyDrive/zones/')

In [66]:
df_join = df_yellow.join(df_zones, df_yellow.PULocationID == df_zones.LocationID)

In [67]:
df_join.registerTempTable('joined_table')

/usr/local/lib/python3.11/dist-packages/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [68]:
q6_query = spark.sql("""
SELECT
    Zone,
    count(1)
FROM
    joined_table
GROUP BY
    1
ORDER BY
    2 ASC
""")

In [69]:
q6_query.show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|Governor's Island...|       1|
|       Rikers Island|       2|
|       Arden Heights|       2|
|         Jamaica Bay|       3|
| Green-Wood Cemetery|       3|
|Charleston/Totten...|       4|
|   Rossville/Woodrow|       4|
|       West Brighton|       4|
|       Port Richmond|       4|
|Eltingville/Annad...|       4|
|         Great Kills|       6|
|        Crotona Park|       6|
|     Mariners Harbor|       7|
|Heartland Village...|       7|
|Saint George/New ...|       9|
|             Oakwood|       9|
|New Dorp/Midland ...|      10|
|       Broad Channel|      10|
|         Westerleigh|      12|
|     Pelham Bay Park|      12|
+--------------------+--------+
only showing top 20 rows



Question 6: Governor's Island/Ellis Island/Liberty Island
